# The Zara Case: What-if Analysis

Install gurobipy and import all the functions in gurobipy.

In [36]:
# Installation
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
# Import all the functions
from gurobipy import *
import numpy as np

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import and prepare the data

In [39]:
# Import the data from an Excel file

# import the library called Pandas first
import pandas as pd

# then, upload the data and save them into "data"
data = pd.read_excel('/content/drive/MyDrive/data_for_what_if.xlsx', index_col=0)

In [40]:
data

,Lille,Paris,Bordeaux,Toulouse,Strasbourg,Lyon,Marseille,Nice,Geneva,Torino,...,Barcelona,Madrid,Bilbao,Valencia,Sevilla,Porto,Lisboa,Capacity,Exten./Construct. Cost,Delivery Cost Budget
Travel Times,,,,,,,,,,,,,,,,,,,,,
Madrid,21.3,18.1,15.1,11.4,23.5,17.7,15.7,18.0,19.7,21.0,...,8.9,0.2,5.7,5.1,6.0,8.1,9.0,123200.0,0.1,4500.0
Lille,0.1,3.2,11.4,12.8,7.5,9.9,14.3,16.5,10.7,14.2,...,17.9,21.2,16.2,22.7,27.1,25.7,27.9,88000.0,0.1,NaN
Orléans,5.0,1.9,6.6,7.9,8.4,6.6,10.8,13.1,7.7,11.1,...,13.1,16.4,11.4,17.9,22.3,20.9,23.1,112000.0,0.5,NaN
Montélimar,11.9,8.7,9.0,5.6,9.1,2.1,2.4,4.6,4.2,5.5,...,7.1,15.6,11.8,11.9,19.6,21.3,23.5,73600.0,0.1,NaN
Châlon,8.1,4.9,8.6,8.6,5.2,1.8,6.3,8.5,2.9,6.3,...,10.9,18.3,13.3,15.8,23.4,22.8,25.0,64000.0,0.2,NaN
Piacenza,16.2,13.2,14.8,12.6,7.8,7.0,7.5,4.6,5.6,2.6,...,14.1,22.6,18.9,18.9,26.6,28.3,30.5,137600.0,0.5,NaN
Bordeaux,15.1,8.4,0.1,3.5,13.8,7.9,9.2,11.5,10.0,12.3,...,9.1,9.8,4.8,11.1,15.7,14.3,16.4,50000.0,4000.0,NaN
Barcelone,17.9,14.8,9.1,5.6,16.1,9.1,7.2,9.5,11.2,12.5,...,0.2,8.9,8.7,5.0,12.7,16.6,17.8,50000.0,15000.0,NaN
Rome,23.1,20.1,21.3,17.8,15.1,13.9,12.8,9.8,12.5,9.5,...,19.3,27.8,24.1,24.2,31.8,33.6,35.7,50000.0,20000.0,NaN


In [41]:
# Select the times, demand, capacity, cost, budget from the excel file

demand=data.iloc[-1,:-3]
time=data.iloc[:-1,:-3]
capacity=data.iloc[:-1,-3]
cost=data.iloc[:-1,-2]
budget=data.iloc[0,-1]


# The number of Distribution Centers (DCs)is equal to the number of rows
num_centers = len(time.index)

# The number of aggregated stores is equal to the number of columns
num_cities = len(time.columns) 

In [42]:
print(num_cities)

21


In [43]:
print(num_centers)

9


## Question 3: **What-if analysis:** Show what the multi-source solution becomes for a set of alternative scenarios that look relevant to you (change some data).

###Case 1: The extension can provide between 10000 and 13000 extra items instead of between 8000 and 13000 items

Initializing the model and defining the decision variables

In [44]:
# Initialize the model
model3_1 = Model('zara-Q3_1')

# Create the variable x[i] = 1 if DC i will be extended or constructed
x = model3_1.addVars(num_centers, vtype=GRB.BINARY, name='extension_construct')

# create the variable y[i] to compute the extension unit if i in range(0,6)
y = model3_1.addVars(6, vtype=GRB.INTEGER, name='extension_unit')

# Create the variable quan[i,j] which denotes the quantity served to city j is from DC i
quant = model3_1.addVars(num_centers, num_cities, vtype=GRB.INTEGER, name='quantity_deliver_site')

In [45]:
# Read in the constants

# m[i] is the cost of construction or extension
m = np.array([i for i in cost])
print('cost: ',m)

# t[i,j] is the delivery time from i to j
t = np.array([[j for j in i] for i in time.values])
print('Dcs and Cities: ',t.shape)

# c[i] is the capacity of sites
c = np.array([i for i in capacity])
print('capacities: ',c)

# d[j] is the demand of cities
d = np.array([i for i in demand])
print('demands: ',d)

cost:  [1.0e-01 1.0e-01 5.0e-01 1.0e-01 2.0e-01 5.0e-01 4.0e+03 1.5e+04 2.0e+04]
Dcs and Cities:  (9, 21)
capacities:  [123200.  88000. 112000.  73600.  64000. 137600.  50000.  50000.  50000.]
demands:  [ 20600.  45000.  21700.  22600.  13500.  39200.  30300.  18500.   9900.
  15800.  23600.   6700.  50200.  18300.  86900. 108600.  19900.  20100.
  26100.  32500.  35000.]


Adding all the constraints in the model

In [46]:
# Constraint 1: Zara can construct or extend no more than 3 sites
model3_1.addConstr(quicksum(x[i] for i in range(num_centers)) <= 3) 

# Constraint 2: If Zara decides to extend, then 10000<= yi <= 13000, else yi = 0
for i in range(6):
    model3_1.addConstr(y[i] >= 10000*x[i])
model3_1.addConstrs(y[i] <= 13000*x[i] for i in range(6))

# Constraint 3: Multiple source supplying. The summation of the supplies from multiple sources for a city should be greater than its demand
for j in range(num_cities):
    model3_1.addConstr(quicksum(quant[i,j] for i in range(num_centers)) >= demand[j])

# Constraint 4: The total monthly delivery costs do not exceed a budget of 4500 euros
model3_1.addConstr(quicksum(quant[i, j] * t[i, j] * 0.001 for i in range(num_centers) for j in range(num_cities)) <= 4500)


# Constraint 5: The supplies must exceed the demands
model3_1.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]+y[i] for i in range(6)) 
model3_1.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]*x[i] for i in range(6,9))

model3_1.update()


Set the objective function and call the solver

In [47]:
# Objective: 
model3_1.setObjective(quicksum(m[i]*y[i] for i in range(6))+quicksum(m[i]*x[i] for i in range(6,9)), GRB.MINIMIZE)

In [48]:
# Call the solver 
model3_1.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 44 rows, 204 columns and 609 nonzeros
Model fingerprint: 0x67b59ee2
Variable types: 0 continuous, 204 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e-04, 5e+04]
  Objective range  [1e-01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
Presolve time: 0.00s
Presolved: 44 rows, 204 columns, 609 nonzeros
Variable types: 0 continuous, 204 integer (9 binary)

Root relaxation: objective 5.660000e+03, 104 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5660.00000    0    5          - 5660.00000      -     -    0s
H    0     0                    6300.0000000 5660

Now we have obtained the solutions for the optimization problem. Displaying the optimal value of the extension/construction cost.

In [49]:
# Save the optimal value of the extension/construction cost
optimal_value_Q3_1 = pd.DataFrame([model3_1.objVal])
optimal_value_Q3_1.index.name = "optVal"

# Display the optimal value the extension/construction cost
print("The optimal value for the extension/construction cost is:")
print(model3_1.objVal)

The optimal value for the extension/construction cost is:
6000.0


Display which DCs need to be extended/constructed (binary values)

In [50]:
# variable: x
# start creating a dataframe of 0 values with the right size (1 row, as many columns as the number of sites)
optimal_x_Q3_1 = pd.DataFrame(0, index=range(1), columns=range(num_centers))
optimal_x_Q3_1.index.name = "x"
# then, iterate on the columns of the dataframe using a for cycle
for i in range(num_centers):
    # and update the value of the column
    optimal_x_Q3_1.iloc[0,i] = x[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_x_Q3_1)

   0  1  2  3  4  5  6  7  8
x                           
0  1  1  0  0  0  0  1  0  0


Display by how much capacity does a DC need to be extended, if it needs to be extended (integer value)

In [51]:
# variable: y
# start creating a dataframe of 0 values with the right size (1 row, with 6 columns)
optimal_y_Q3_1 = pd.DataFrame(0, index=range(1), columns=range(6))
optimal_y_Q3_1.index.name = "y"
# then, iterate on the columns of the dataframe using a for cycle
for i in range(6):
    # and update the value of the column
    optimal_y_Q3_1.iloc[0,i] = y[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_y_Q3_1)

       0      1  2  3  4  5
y                          
0  10000  10000  0  0  0  0


In [52]:
# Report the optimal solution in the excel file

# to write in the excel file you need to use the following line of code 
# in which you open the excel file called 'Speedy_Pizza_Data.xlsx' in writer mode
with pd.ExcelWriter('/content/drive/MyDrive/data_for_what_if.xlsx', mode='a', if_sheet_exists='overlay') as writer:
# then, you simply upload the dataframes a new sheet called 'Q1', one under the other
    optimal_value_Q3_1.to_excel(writer, sheet_name='Q3_1')
    optimal_x_Q3_1.to_excel(writer, sheet_name='Q3_1', startrow=3)
    optimal_y_Q3_1.to_excel(writer, sheet_name='Q3_1', startrow=6)

In [54]:
#Visualization for the average delivery time for all the stores
sum=[0]*21
sum_quant_across_rows = [0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum_quant_across_rows[j]=sum_quant_across_rows[j]+quant_values[i,j]
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]
  sum[j]=sum[j]/sum_quant_across_rows[j]

print("The average delivery time for all the stores are (What-if case 1):")
print(sum)

The average delivery time for all the stores are (What-if case 1):
[0.1, 3.2, 6.6, 7.9, 7.5, 1.8, 5.075247524752475, 4.6, 2.9, 2.6, 1.0, 3.5, 7.4, 10.1, 10.120271576524742, 4.1618674033149174, 11.4, 11.9, 6.0, 20.9, 9.0]


In [55]:
#Visualization for the average delivery time from all the DCs
sum=[0]*9
sum_quant_across_columns = [0]*9

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for i in range(9):
  for j in range (21):
    sum_quant_across_columns[i]=sum_quant_across_columns[i]+quant_values[i,j]
    sum[i] = sum[i]+quant_values[i,j]*t[i,j]
  sum[i]=sum[i]/sum_quant_across_columns[i]

print("The average delivery time from all the DCs are (What-if case 1):")
print(sum)

The average delivery time from all the DCs are (What-if case 1):
[3.693038711001954, 5.7635745220019325, 13.036891071428572, 7.714809782608696, 3.0178125, 5.547165697674418, 9.499574, nan, nan]


<ipython-input-55-4b5cb6326899>:11: RuntimeWarning: invalid value encountered in double_scalars
  sum[i]=sum[i]/sum_quant_across_columns[i]


In [56]:
#Visualization for the Monthly delivery costs for all the stores

sum=[0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]*0.001

print("The average Monthly delivery costs for all the stores are (What-if case 1):")
print(sum)

The average Monthly delivery costs for all the stores are (What-if case 1):
[2.06, 144.0, 143.22, 178.54, 101.25, 70.56, 153.78, 85.10000000000001, 28.71, 41.08, 23.6, 23.45, 371.48, 184.83, 879.4516000000001, 451.9788000000001, 226.86, 239.19, 156.6, 679.25, 315.0]


**Comments**: If the extension can provide between 10000 and 13000 extra items instead of between 8000 and 13000 items, the DC 1 and 2 needs to be extended each by 10000 units, while DC 7 needs to be constructed. The optimization results in the minimum extension/construction cost of 6000 euros.

###Case 2: The monthly delivery cost to an aggregated store do not exceed a budget of 3500 euros instead of 4500 euros

Initializing the model and defining the decision variables

In [57]:
# Initialize the model
model3_2 = Model('zara-Q3_2')

# Create the variable x[i] = 1 if DC i will be extended or constructed
x = model3_2.addVars(num_centers, vtype=GRB.BINARY, name='extension_construct')

# create the variable y[i] to compute the extension unit if i in range(0,6)
y = model3_2.addVars(6, vtype=GRB.INTEGER, name='extension_unit')

# Create the variable quan[i,j] which denotes the quantity served to city j is from DC i
quant = model3_2.addVars(num_centers, num_cities, vtype=GRB.INTEGER, name='quantity_deliver_site')

Defining the input parameters

In [58]:
# Read in the constants

# m[i] is the cost of construction or extension
m = np.array([i for i in cost])
print('cost: ',m)

# t[i,j] is the delivery time from i to j
t = np.array([[j for j in i] for i in time.values])
print('Dcs and Cities: ',t.shape)

# c[i] is the capacity of sites
c = np.array([i for i in capacity])
print('capacities: ',c)

# d[j] is the demand of cities
d = np.array([i for i in demand])
print('demands: ',d)

cost:  [1.0e-01 1.0e-01 5.0e-01 1.0e-01 2.0e-01 5.0e-01 4.0e+03 1.5e+04 2.0e+04]
Dcs and Cities:  (9, 21)
capacities:  [123200.  88000. 112000.  73600.  64000. 137600.  50000.  50000.  50000.]
demands:  [ 20600.  45000.  21700.  22600.  13500.  39200.  30300.  18500.   9900.
  15800.  23600.   6700.  50200.  18300.  86900. 108600.  19900.  20100.
  26100.  32500.  35000.]


Defining all the constraints

In [59]:
# Constraint 1: Zara can construct or extend no more than 3 sites
model3_2.addConstr(quicksum(x[i] for i in range(num_centers)) <= 3) 

# Constraint 2: If Zara decides to extend, then 8000<= yi <= 13000, else yi = 0
for i in range(6):
    model3_2.addConstr(y[i] >= 8000*x[i])
model3_2.addConstrs(y[i] <= 13000*x[i] for i in range(6))

# Constraint 3: Multiple source supplying. The summation of the supplies from multiple sources for a city should be greater than its demand
for j in range(num_cities):
    model3_2.addConstr(quicksum(quant[i,j] for i in range(num_centers)) >= d[j])

# Constraint 4: The total monthly delivery costs of each aggregated stores do not exceed a budget of 3500 euros
model3_2.addConstr(quicksum(quant[i, j] * t[i, j] * 0.001 for i in range(num_centers) for j in range(num_cities)) <= 3500)


# Constraint 5: The supplies must exceed the demands
model3_2.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]+y[i] for i in range(6)) 
model3_2.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]*x[i] for i in range(6,9))

model3_2.update()


Set the objective function and call the solver

In [60]:
# Objective: 
model3_2.setObjective(quicksum(m[i]*y[i] for i in range(6))+quicksum(m[i]*x[i] for i in range(6,9)), GRB.MINIMIZE)

In [61]:
# Call the solver 
model3_2.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 44 rows, 204 columns and 609 nonzeros
Model fingerprint: 0xcce20d08
Variable types: 0 continuous, 204 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e-04, 5e+04]
  Objective range  [1e-01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
Presolve time: 0.00s
Presolved: 44 rows, 204 columns, 609 nonzeros
Variable types: 0 continuous, 204 integer (9 binary)

Root relaxation: objective 2.244764e+04, 104 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 22447.6419    0   17          - 22447.6419      -     -    0s
     0     0 39000.0000    0    2          - 3900

Now we have obtained the solutions for the optimization problem. Displaying the optimal value of the extension/construction cost.

In [62]:
# Save the optimal value of the extension/construction cost
optimal_value_Q3_2 = pd.DataFrame([model3_2.objVal])
optimal_value_Q3_2.index.name = "optVal"

# Display the optimal value the extension/construction cost
print("The optimal value for the extension/construction cost is:")
print(model3_2.objVal)

The optimal value for the extension/construction cost is:
39000.0


Display which DCs need to be extended/constructed (binary values)

In [63]:
# variable: x
# start creating a dataframe of 0 values with the right size (1 row, as many columns as the number of sites)
optimal_x_Q3_2 = pd.DataFrame(0, index=range(1), columns=range(num_centers))
optimal_x_Q3_2.index.name = "x"
# then, iterate on the columns of the dataframe using a for cycle
for i in range(num_centers):
    # and update the value of the column
    optimal_x_Q3_2.iloc[0,i] = x[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_x_Q3_2)

   0  1  2  3  4  5  6  7  8
x                           
0  0  0  0  0  0  0  1  1  1


Display by how much capacity does a DC need to be extended, if it needs to be extended (integer value)

In [64]:
# variable: y
# start creating a dataframe of 0 values with the right size (1 row, with 6 columns)
optimal_y_Q3_2 = pd.DataFrame(0, index=range(1), columns=range(6))
optimal_y_Q3_2.index.name = "y"
# then, iterate on the columns of the dataframe using a for cycle
for i in range(6):
    # and update the value of the column
    optimal_y_Q3_2.iloc[0,i] = y[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_y_Q3_2)

   0  1  2  3  4  5
y                  
0  0  0  0  0  0  0


In [65]:
# Report the optimal solution in the excel file

# to write in the excel file you need to use the following line of code 
# in which you open the excel file called 'Speedy_Pizza_Data.xlsx' in writer mode
with pd.ExcelWriter('/content/drive/MyDrive/data_for_what_if.xlsx', mode='a', if_sheet_exists='overlay') as writer:
# then, you simply upload the dataframes a new sheet called 'Q1', one under the other
    optimal_value_Q3_2.to_excel(writer, sheet_name='Q3_2')
    optimal_x_Q3_2.to_excel(writer, sheet_name='Q3_2', startrow=3)
    optimal_y_Q3_2.to_excel(writer, sheet_name='Q3_2', startrow=6)

In [69]:
#Visualization for the average delivery time for all the stores
sum=[0]*21
sum_quant_across_rows = [0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum_quant_across_rows[j]=sum_quant_across_rows[j]+quant_values[i,j]
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]
  sum[j]=sum[j]/sum_quant_across_rows[j]

print("The average delivery time for all the stores are (What-if case 2):")
print(sum)

The average delivery time for all the stores are (What-if case 2):
[0.1, 2.758, 6.6, 7.935294117647059, 7.5, 1.8, 4.163366336633663, 4.6, 2.9, 2.6, 1.0, 3.5, 3.357525896414343, 3.2, 3.1299194476409666, 0.2, 11.4, 11.9, 15.7, 20.9, 13.313142857142857]


In [70]:
#Visualization for the average delivery time from all the DCs
sum=[0]*9
sum_quant_across_columns = [0]*9

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for i in range(9):
  for j in range (21):
    sum_quant_across_columns[i]=sum_quant_across_columns[i]+quant_values[i,j]
    sum[i] = sum[i]+quant_values[i,j]*t[i,j]
  sum[i]=sum[i]/sum_quant_across_columns[i]

print("The average delivery time from all the DCs are (What-if case 2):")
print(sum)

The average delivery time from all the DCs are (What-if case 2):
[1.2428571428571429, 3.1089341692789967, 11.222678571428572, 7.350815217391304, 3.0609375, 3.8646593518754786, 16.00709677419355, 0.2, 1.4319909560172348]


In [71]:
#Visualization for the Monthly delivery costs for all the stores

sum=[0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]*0.001

print("The average Monthly delivery costs for all the stores are (What-if case 2):")
print(sum)

The average Monthly delivery costs for all the stores are (What-if case 2):
[2.06, 124.11000000000001, 143.22, 188.85999999999999, 101.25, 70.56, 126.15, 85.10000000000001, 28.71, 41.08, 23.6, 23.45, 168.54780000000002, 58.56, 271.99, 21.72, 226.86, 239.19, 409.77, 679.25, 465.96000000000004]


**Comments**: If the monthly delivery cost to an aggregated store cannot do not exceed a budget of 3500 euros instead of 4500 euros, then DCs 7, 8 and 9 needs to be contructed. No extensions in existing DCs are necessary. The optimization results in the minimum extension/construction cost of 39000 euros.